# Goal Oriented Generation 

Based on prompts from [Creating Suspenseful Stories with Large Language Models](https://arxiv.org/abs/2402.17119).

In [20]:
from dotenv import load_dotenv

load_dotenv()

True

In [21]:
from langchain_aws import ChatBedrock
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage

model = ChatBedrock(
    model_id="anthropic.claude-3-haiku-20240307-v1:0",
    model_kwargs={"temperature": 0.1},
)

role = """You are a short story writer with a passion for creative writing and storytelling. 
Your task is to create engaging stories that captivate your audience and leave them wanting more. 
"""

def prompt_from_template(template: str):
    return ChatPromptTemplate.from_messages([
        ("system", role),
        ("human", template)
    ])


In [22]:
genre = "Espionage Thriller"
name_pormpt = prompt_from_template(template="Make up the name and the occupation of a protagonist of a {genre}.")

name_chain = name_pormpt | model | StrOutputParser()
name_occupation = name_chain.invoke({"genre":genre})
print(name_occupation)

Here is the name and occupation of a protagonist for an espionage thriller:

Name: Evelyn Sinclair
Occupation: International Intelligence Operative

Evelyn Sinclair is a highly skilled and seasoned intelligence operative who works for a covert government agency. She is an expert in counterintelligence, surveillance, and strategic operations. Evelyn has been tasked with infiltrating a dangerous global crime syndicate that is threatening national security. With her sharp instincts, quick thinking, and mastery of disguise and deception, Evelyn must navigate a web of lies and betrayal to uncover the syndicate's sinister plans before it's too late. Her mission will take her across the globe, from the shadowy back alleys of Eastern Europe to the high-stakes boardrooms of multinational corporations. Evelyn must use all of her cunning and resourcefulness to stay one step ahead of the enemy and prevent a catastrophic attack. The fate of the free world hangs in the balance, and Evelyn is the onl

In [23]:


goal_prompt = prompt_from_template(template="<name_occupation>{name_occupation}</name_occupation>\n\nGiven the <name_occupation>, create a very concrete goal the protagonist wants to achieve")

gaol_chain = goal_prompt | model | StrOutputParser()
goal = gaol_chain.invoke({"name_occupation":name_occupation})
print(goal)

Given the details provided about the protagonist, Evelyn Sinclair, an international intelligence operative, her concrete goal in this espionage thriller would be:

To infiltrate the dangerous global crime syndicate, uncover their sinister plans, and prevent a catastrophic attack that threatens national security and the free world.

Evelyn's mission is high-stakes and the consequences of failure are dire. She must use all of her skills, expertise, and resourcefulness to navigate the web of lies and betrayal, stay one step ahead of the enemy, and ultimately dismantle the crime syndicate before they can carry out their devastating plot. The fate of the world rests on Evelyn's shoulders, and she is driven to succeed in this critical operation.


In [26]:

situtaion = """<story_background>
<name_occupation>{name_occupation}</cname_occupation> 
<goal>{goal}</goal>
</story_background>
Tell me about a dire situation that the protagonist would be put in, if the protagonist couldn’t achieve the goal."""

situtaion_prompt = prompt_from_template(template=situtaion)

situation_chain = situtaion_prompt | model | StrOutputParser()
dire_situation = situation_chain.invoke({"name_occupation":name_occupation, "goal":goal})

print(dire_situation)

If Evelyn Sinclair, the skilled intelligence operative, fails to infiltrate the global crime syndicate and uncover their sinister plans, the consequences could be catastrophic:

The crime syndicate's attack would likely be a large-scale, coordinated strike that could have devastating effects on national security and the stability of the free world. Without Evelyn's intervention, the syndicate may be able to detonate a series of strategic terrorist attacks, crippling critical infrastructure, disrupting global financial markets, or even deploying weapons of mass destruction.

The loss of life from such an attack would be staggering, and the resulting chaos and panic could lead to the collapse of governments, the outbreak of war, and the unraveling of the international order that has maintained global peace and prosperity. Evelyn's failure would embolden the syndicate and empower the forces of darkness, ushering in a new era of lawlessness, oppression, and unchecked criminal power.

Evely

In [27]:
intro_instruction =  """<story_background>
<name_occupation>{name_occupation}</cname_occupation> 
<goal>{goal}</goal>
<situation>{dire_situation}</situation>
</story_background>
Write an introductory paragraph of a story that covers all the above information.
"""

intro_prompt = prompt_from_template(template=intro_instruction)
intro_chain = intro_prompt | model | StrOutputParser()
intro = intro_chain.invoke({"name_occupation":name_occupation, "goal":goal, "dire_situation":dire_situation})
print(intro)

Here is an introductory paragraph for the espionage thriller story featuring Evelyn Sinclair, the skilled international intelligence operative:

The fate of the free world rested on Evelyn Sinclair's shoulders. As a seasoned operative for a covert government agency, she had navigated the shadowy world of espionage and counterintelligence with unparalleled skill. But her latest mission was unlike any she had faced before. Evelyn had been tasked with infiltrating a dangerous global crime syndicate, one that threatened to unleash a catastrophic attack that could cripple nations and unravel the very fabric of international order. With her sharp instincts, quick thinking, and mastery of disguise and deception, Evelyn knew she was the only one capable of uncovering the syndicate's sinister plans before it was too late. The fate of the world hung in the balance, and failure was not an option. Evelyn steeled her resolve, knowing that she must succeed in this high-stakes operation, no matter th